# Import Data

In [1]:
import os
import pandas as pd
import numpy as np

# Import data from files
os.chdir('/Users/talhajamal/Desktop/Code/BigData1Project')
data = pd.read_csv("data/Returns_Data.csv")
data['date'] = pd.to_datetime(data['date'], dayfirst=False)
characteristics = pd.read_csv("data/Stock_Characteristics_Data.csv")
dictionary = pd.read_excel("data/StockDataDictionary.xlsx")

# Create new dataframes
prices = data.pivot(index='date', columns='ticker', values='PRC')
volume = data.pivot(index='date', columns='ticker', values='VOL')
returns = data.pivot(index='date', columns='ticker', values='RET')
#returns = returns * 100 # Scale returns to percentage
# Summary of Returns
returns_summary = returns.describe()
shares_outstanding = data.pivot(index='date', columns='ticker', values='SHROUT')
value_weighted_returns = data.pivot(index='date', columns='ticker', values='vwretd')
equal_weighted_returns = data.pivot(index='date', columns='ticker', values='ewretd')
tickers = prices.columns # List of Tickers

# Functions

In [51]:
def portfolio_performance(weights, returns):
    """ Calculate Portfolio Performance"""
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    annualizedReturns = np.sum(meanReturns*weights)*252
    annualizedStd = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(252)
    return round(annualizedReturns*100, 2), round(annualizedStd*100, 2)


# Test Functions

In [55]:
weights = np.array([1/100 for x in returns])
equalWeightedRet, equalWeightedVol = portfolio_performance(weights=weights, returns=returns)
print("Equal Weighted Portfolio Returns: ", equalWeightedRet, "%")
print("Equal Weighted Portfolio Volatility: ", equalWeightedVol, "%")

Equal Weighted Portfolio Returns:  15.93 %
Equal Weighted Portfolio Volatility:  18.62 %
